In [2]:
from utils.get_setting import setting_data, print_setting_data, find_key_path, value_of_key

crawler_RDF_folder_path = find_key_path("碩博士論文_RDF")
apply_list_folder_path = find_key_path("統計表分析") 
recommendation_list_folder_path = find_key_path("產學申請名冊") 

print(f"(fine_key_path) CHROMA:{find_key_path('CHROMA')}")

(fine_key_path) CHROMA:./database/chroma_database


In [11]:
import pandas as pd

def split_institution(department_full):
    keywords = ['大學', '院', '博物館', '學校', '法人']  # 列出所有可能的分割關鍵字
    for keyword in keywords:
        if keyword in department_full:
            school, department = department_full.split(keyword, 1)
            school += keyword  # 將關鍵字加回學校名稱中
            return pd.Series([school.strip(), department.strip()])
    return pd.Series([department_full.strip(), ''])  # 如果沒有關鍵字，就只有學校沒有系所

print(split_institution("國立雲林科技大學前瞻學士學位學程"))

0    國立雲林科技大學
1    前瞻學士學位學程
dtype: object


In [10]:
import re

def extract_text_in_parentheses(text):
    if isinstance(text, str):
        if text == "":
            return [['', '']]
        
        # 捕捉名字和括號內的內容
        pattern = r'([^;]+)\(([^)]+)\)' 
        matches = re.findall(pattern, text)
        
        if matches:  details = [[match[0].strip(), match[1]] for match in matches] # 成對的
        else:
            # 如果沒有匹配到括號中的內容，檢查是否是部門名稱
            if "大學" in text or "學系" in text or "研究所" in text:
                details = [["", text.strip()]]
            else:
                details = [[text.strip(), '']]
        
        return details
    
    elif isinstance(text, list):
        result = []
        for item in text:
            result.extend(extract_text_in_parentheses(item))
        return result

    return []

# 使用範例
text = "薛雅馨(國立雲林科技大學電子工程系暨研究所)"
result = extract_text_in_parentheses(text)
print(result)

# 使用範例
text = """
孫敏德(國立中央大學資訊工程學系);
張克士(國立暨南國際大學教育學院);
洪炯宗(國立中央大學資訊工程學系)
"""
result = extract_text_in_parentheses(text)
print(result)

new_text_list = ['吳炳飛', "薛雅馨(國立雲林科技大學電子工程系暨研究所)"]
result = extract_text_in_parentheses(new_text_list)
print(result) # 

only_department = ['國立陽明交通大學電控工程研究所', '國立清華大學電機工程學系（所）']
result = extract_text_in_parentheses(only_department)
print(result) # 

[['薛雅馨', '國立雲林科技大學電子工程系暨研究所']]
[['孫敏德', '國立中央大學資訊工程學系'], ['張克士', '國立暨南國際大學教育學院'], ['洪炯宗', '國立中央大學資訊工程學系']]
[['吳炳飛', ''], ['薛雅馨', '國立雲林科技大學電子工程系暨研究所']]
[['', '國立陽明交通大學電控工程研究所'], ['', '國立清華大學電機工程學系（所）']]


In [7]:
[department for name, department in result]

['', '國立雲林科技大學電子工程系暨研究所']

In [9]:
from utils.get_setting import find_key_path

person = "方文聘"
crawler_RDF_folder_path = find_key_path("碩博士論文_RDF")
crawler_RDF_data = pd.read_excel(crawler_RDF_folder_path)

def find_crawler_person_relative_school(person, crawler_RDF_data):
    person_data = crawler_RDF_data[crawler_RDF_data['學生姓名'] == person]
    
    if len(person_data) == 0:
        return ""
    else:
        result_list = []
        for department in person_data['畢業學校']:
            print(department)
            result_list.append(department.split("／")[0])
            
        return list(set(result_list))
    
find_crawler_person_relative_school(person, crawler_RDF_data)

國立交通大學／機械工程研究所
國立交通大學／資訊科學與工程研究所


['國立交通大學']

In [21]:
from itertools import chain

text = """
孫敏德(國立中央大學資訊工程學系);
張克士(國立暨南國際大學教育學院);
洪炯宗(國立中央大學資訊工程學系)
"""
result = extract_text_in_parentheses(text)
data = list([find_crawler_person_relative_school(name, crawler_RDF_data) for name, department in result]),


from itertools import chain

# data = (['', '', ['國立臺灣大學']],)
flattened_data = list(chain.from_iterable(chain.from_iterable(data)))

print(flattened_data)


國立臺灣大學／資訊科學研究所
國立臺灣大學／資訊工程研究所
['國立臺灣大學']


In [6]:
def filter_committee_advanced(schools_info, committee_members, filter_pairs):
    """
    進階過濾委員名單，根據具體的配對關係進行過濾，並提供過濾的具體原因。

    :param schools_info: 包含學校相關資訊的字典
    :param committee_members: 包含委員相關資訊的列表
    :param filter_pairs: 列表，包含過濾配對條件，例如 [("申請學校", "就職學校")]
    :return: 一個字典，包含過濾前後的委員名單和未過濾的委員名單，以及過濾原因
    """
    
    filtered_members = set()
    filter_reasons = {}

    # 根據配對條件進行過濾
    for school_type, member_field in filter_pairs:
        if school_type in schools_info and schools_info[school_type]:
            school_list = schools_info[school_type] if isinstance(schools_info[school_type], list) else [schools_info[school_type]]
            for member in committee_members:
                matching_schools = [school for school in member[member_field] if school in school_list and school]
                if matching_schools:
                    filtered_members.add(member['委員名稱'])
                    filter_reasons[member['委員名稱']] = f"{school_type} 與 {member_field} ({', '.join(matching_schools)}) 重疊"

    # 創建過濾後的委員名單
    remaining_members = [member['委員名稱'] for member in committee_members if member['委員名稱'] not in filtered_members]

    # 返回結果
    return {
        'Filtered Members': list(filtered_members),
        'Remaining Members': remaining_members,
        'Filter Reasons': filter_reasons
    }

# 測試修正後的函數
filter_pairs = [('計畫申請學校', '委員曾就職學校'), ('共同計畫主持的學校', '委員曾就職學校')]
schools_info =  {'計畫申請學校': '國立成功大學', '共同計畫主持的學校': [''], '計畫主持人過去畢業的學校': [], '共同主持人過去的畢業學校': ['淡江大學']}
committee_members =  [
    {'委員名稱': '蘇明祥', '委員曾就職學校': ['東吳大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '羅友聲', '委員曾就職學校': [], '委員過去畢業學校': []}, 
    {'委員名稱': '顏孟華', '委員曾就職學校': ['國立成功大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '黃建宏', '委員曾就職學校': ['國立成功大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '鐘玉芳', '委員曾就職學校': ['東海大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '郭景明', '委員曾就職學校': ['國立成功大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '龔旭陽', '委員曾就職學校': ['國立屏東科技大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '簡廷因', '委員曾就職學校': ['元智大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '連琬菁', '委員曾就職學校': ['國立臺灣大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '徐建業', '委員曾就職學校': ['國立臺北護理健康大學'], '委員過去畢業學校': []}
]

result = filter_committee_advanced(schools_info, committee_members, filter_pairs)
for item in result:
    print(f"{item}: {result[item]}")

print(result)
print("---------")

schools_info = {'計畫申請學校': '', '共同計畫主持的學校': ['', '國立陽明交通大學'], '計畫主持人過去畢業的學校': [], '共同主持人過去畢業的學校': []}
committee_members =[
    {'委員名稱': '鄭昌杰', '委員曾就職學校': ['國立陽明交通大學', '國立交通大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '彭其瀚', '委員曾就職學校': ['國立陽明交通大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '黃怡錚', '委員曾就職學校': ['元智大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '石昭玲', '委員曾就職學校': ['中華大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '施皇嘉', '委員曾就職學校': ['元智大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '周建興', '委員曾就職學校': ['淡江大學', '國立臺北教育大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '鍾國亮', '委員曾就職學校': ['國立臺灣科技大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '何前程', '委員曾就職學校': ['國立雲林科技大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '范育成', '委員曾就職學校': ['國立臺北科技大學'], '委員過去畢業學校': []}, 
    {'委員名稱': '王才沛', '委員曾就職學校': ['國立陽明交通大學', '國立交通大學'], '委員過去畢業學校': []}
]

result = filter_committee_advanced(schools_info, committee_members, filter_pairs)
for item in result:
    print(f"{item}: {result[item]}")
print(result)


Filtered Members: ['顏孟華', '郭景明', '黃建宏']
Remaining Members: ['蘇明祥', '羅友聲', '鐘玉芳', '龔旭陽', '簡廷因', '連琬菁', '徐建業']
Filter Reasons: {'顏孟華': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊', '黃建宏': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊', '郭景明': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊'}
{'Filtered Members': ['顏孟華', '郭景明', '黃建宏'], 'Remaining Members': ['蘇明祥', '羅友聲', '鐘玉芳', '龔旭陽', '簡廷因', '連琬菁', '徐建業'], 'Filter Reasons': {'顏孟華': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊', '黃建宏': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊', '郭景明': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊'}}
---------
Filtered Members: ['鄭昌杰', '彭其瀚', '王才沛']
Remaining Members: ['黃怡錚', '石昭玲', '施皇嘉', '周建興', '鍾國亮', '何前程', '范育成']
Filter Reasons: {'鄭昌杰': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊', '彭其瀚': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊', '王才沛': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊'}
{'Filtered Members': ['鄭昌杰', '彭其瀚', '王才沛'], 'Remaining Members': ['黃怡錚', '石昭玲', '施皇嘉', '周建興', '鍾國亮', '何前程', '范育成'], 'Filter Reasons': {'鄭昌杰': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊', '彭其瀚': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊', '王才沛': '共同計畫主

In [10]:
def merge_results(result1, result2):
    """
    合併兩個過濾結果。

    :param result1: 第一個過濾結果字典
    :param result2: 第二個過濾結果字典
    :return: 合併後的過濾結果字典
    """
    # 合併 'Filtered Members'
    merged_filtered_members = list(set(result1['Filtered Members'] + result2['Filtered Members']))
    
    # 合併 'Remaining Members'
    merged_remaining_members = list(set(result1['Remaining Members'] + result2['Remaining Members']) - set(merged_filtered_members))
    
    # 合併 'Filter Reasons'
    merged_filter_reasons = {**result1['Filter Reasons'], **result2['Filter Reasons']}
    
    return {
        'Filtered Members': merged_filtered_members,
        'Remaining Members': merged_remaining_members,
        'Filter Reasons': merged_filter_reasons
    }

# 使用範例
result1 = {'Filtered Members': [], 'Remaining Members': ['蘇明祥', '羅友聲', '鐘玉芳', '龔旭陽', '簡廷因', '連琬菁', '徐建業'], 'Filter Reasons': {'顏孟華': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊', '黃建宏': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊', '郭景明': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊'}}
result2 ={'Filtered Members': ['鄭昌杰', '彭其瀚', '王才沛'], 'Remaining Members': ['黃怡錚', '石昭玲', '施皇嘉', '周建興', '鍾國亮', '何前程', '范育成'], 'Filter Reasons': {'鄭昌杰': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊', '彭其瀚': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊', '王才沛': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊'}}

merged_result = merge_results(result1, result2)
for item in merged_result:
    print(f"{item}: {merged_result[item]}")



Filtered Members: ['鄭昌杰', '彭其瀚', '王才沛']
Remaining Members: ['龔旭陽', '范育成', '施皇嘉', '連琬菁', '蘇明祥', '鍾國亮', '鐘玉芳', '黃怡錚', '石昭玲', '羅友聲', '周建興', '簡廷因', '徐建業', '何前程']
Filter Reasons: {'顏孟華': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊', '黃建宏': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊', '郭景明': '計畫申請學校 與 委員曾就職學校 (國立成功大學) 重疊', '鄭昌杰': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊', '彭其瀚': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊', '王才沛': '共同計畫主持的學校 與 委員曾就職學校 (國立陽明交通大學) 重疊'}
